# NEURON: Ball-and-stick electrical model of a neuron

This notebook is under [LGPLv3](https://raw.githubusercontent.com/BlueBrain/MOOC-neurons-and-synapses-2017/master/LICENSE.txt) license.

Some parts of the notebook were adopted from Blue Brain Project / EPFL ©2005-2017.

## 1. Introduction

In this exercise, you will be introduced to the NEURON simulator. You will learn how to:
* **Load** the NEURON simulator in Python using Jupyter Notebook environment
 * You can find additional information about Jupyter Notebooks on [this page](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/index.html).
* Construct a **single compartmental electrical** model
* **Run a simulation**, record the membrane voltage of the model and inject an external current
* **Add ion channels** to the membrane of the model
* Explore the properties of the Hodgkin-Huxley equation-based model of a neuron

Execute the following cell to get the latest version of the software:

In [ ]:
import os
#os.unsetenv('PYTHONHOME') # Solve an issue with NEURON simulator import

# Install software, ignore some warnings
#!pip install -q bluepyopt==1.5.12 matplotlib==2.0.2 numpy==1.13.0 2>&1 | grep -v 'SNIMissingWarning\|InsecurePlatformWarning'

# Show matplotlib plots inline
%matplotlib inline 

## 2. Loading the NEURON simulator

We first import the NEURON Python module

In [ ]:
import neuron

The documentation of NEURON is available [here](https://www.neuron.yale.edu/neuron/docs).

Historically the NEURON simulator was controlled using the [HOC language](https://www.neuron.yale.edu/neuron/static/new_doc/programming/hocsyntax.html).
More and more scientists are switching to the Python interface of NEURON. In Python there is something called the [HOCObject](https://www.neuron.yale.edu/neuron/static/docs/help/neuron/neuron/classes/python.html#HocObject) which can be accessed by 'h' property of the neuron module, and which contains the HOC functionality of NEURON. For example, to load the 'stdrun.hoc' file which contains some handy functionality one calls the HOC function load_file using:

In [ ]:
print neuron.h
# Load external files
neuron.h.load_file("/home/jones/hoc/stdrun.hoc");

To make sure we start from a clean state, we initialise the simulator

In [ ]:
neuron.h.stdinit();

It is important to know that all the values used in NEURON have a certain default unit. 
The list of the defaults can be accessed here: https://www.neuron.yale.edu/neuron/static/docs/units/unitchart.html

For now the most important ones are:
* **length**: micrometer (um)
* **time**: milliseconds (ms)
* **voltage**: millivolt (mV)
* **specific capacitance**: microfarad/cm2 (uf/cm2)

## 3. Constructing a neuron consisting of one compartment

The basic structure of a cell in NEURON is a set of interconnected cylinders (called segments or compartments). Each of these cylinders represents an isopotential part of the cell. The segments between the branch points are grouped together in sections.

![Segments](https://bbp.epfl.ch/public/SimNeuroMOOC/images/TutBallStick/segments.jpg)

<center> *Figure 1.* Cable (upper panel) and compartmental (lower panel) representation of a branch of Purkinje cell neuron. Figure adapted from: Schutter, E. D. (2009). [Computational modeling methods for neuroscientists. The MIT Press](https://search.library.brown.edu/catalog/b7926844).</center>

Following a familiar electrical circuit representation of a neuron discussed in class, the somatic membrane of a neuron with a dendrite can be modelled as shown below:

![ecircuit](https://bbp.epfl.ch/public/SimNeuroMOOC/images/TutBallStick/
ballandstick_circuit.jpg)
<center> *Figure 2.* Ball-and-stick model of neuron with a dendrite.</center>

The soma of our cell represents one of the sections.
Let's start by creating a cell with just a soma:

In [ ]:
soma = neuron.h.Section(name='soma')

print "Soma object:", soma
print "Soma object name: ", soma.name()

print "Number of segments in the soma:", soma.nseg

Just as every other section in NEURON, the soma is represented as a cylinder. We can set the length and diameter of this section.

In [ ]:
soma.L = 40
soma.diam = 40
soma.Ra = 125 # Ohm * cm
print "Soma length: %f micron" % soma.L 
print "Soma diameter: %f micron" % soma.diam
print "Soma axial resistance: %f Ohm x cm" % soma.Ra

We can calculate the surface area of the soma using the 'area()' function, and compare it to the equation to calculate the surface area of a cylinder:

In [ ]:
soma_area_eq = 2 * neuron.h.PI * soma.L * soma.diam / 2
print "Soma area according to cylinder surface area equation: %f micron^2" % soma_area_eq

# The 0.5 refers to the segment in the middle of the soma
# Because there is only one segment, in this case it refers to the entire soma
soma_area = neuron.h.area(0.5, sec=soma)
print "Soma area according to NEURON: %f micron^2" % soma_area

print "Both values match: %s" % (soma_area_eq == soma_area)

So now we have defined the shape of this single compartmental cell. Electrically the only component that is present for now is the capacitance of the cell membrane. We can retrieve or set the specific membrane capacitance by accessing the 'cm' field. The units of this field are microFarad per cm2. You will see that in NEURON most quantities related to the surface currents are specified per membrane surface area.

In [ ]:
print "Specific capacitance: %f uf/cm2" % soma.cm

## 4. Running a simulation

Let's now run a simulation over time. We will record the [membrane voltage](http://www.st-andrews.ac.uk/~wjh/neurotut/mempot.html) in the middle of the soma during the simulation:

In [ ]:
print "Membrane voltage soma: %f mV" % soma(.5).v # mV

As you can see, the voltage of the cell has been initialised to -65 mV, which roughly corresponds to a typical value in neurons at rest (also called the *resting membrane potential*).

The simulator will integrate the equations over time. The current time is saved in a variable called 't':

In [ ]:
print "Current time: %f ms" % neuron.h.t # ms

When the run() function is called, the simulation will run until a predefined time stored in the the 'tstop' variable. Every timestep, defined by 'dt', a new voltage value will be calculated. 

In [ ]:
neuron.h.tstop = 100
print "Simulation stop time: %f ms" % neuron.h.tstop
print "Integration time step: %f ms" % neuron.h.dt

To record the time and voltage we create two NEURON Vector objects. These objects have a 'record' method that, when passed a reference to a variable, will record the value of that variable in the vector during the simulation.

In [ ]:
time = neuron.h.Vector()
voltage = neuron.h.Vector()

time.record(neuron.h._ref_t)
voltage.record(soma(.5)._ref_v);

In [ ]:
neuron.h.run()

def plot_tv(time_array, voltage_array, show=True, label=None, constants=[]):
    import matplotlib.pyplot as plt
    import numpy
    plt.plot(time_array, voltage_array, label=label)
    for constant in constants:
        plt.plot(time_array, constant*numpy.ones(len(time_array)))
    plt.xlabel('Time (ms)')
    plt.ylabel('Membrane voltage (mV)')
    if show:
        plt.show()
    
plot_tv(time, voltage)

So, basically the voltage has stayed the same over time. We can see why this happened by looking at the following equation NEURON is integrating:

$$C_{m} \frac{dV}{dt} = I_{m}$$

Basically, the change in voltage is related to the current going across the membrane. Since we don't have any mechanisms that allow currents to cross the membrane, $I_{m}$ is zero, and our voltage stay the same:

$$dV = I_{m} \frac{dt}{C_{m}} = 0 \frac{dt}{C_{m}}$$

## 5. Injecting a current

Let's now see how we can charge the cell membrane voltage. For this, we stick a virtual electrode in the center of the soma, and inject current using a current clamp. We specify an amplitude (in nA), a delay and a duration for our stimulus.

In [ ]:
iclamp = neuron.h.IClamp(.5, sec=soma)
iclamp.amp = 0.5 # nA
iclamp.delay = 10 # ms
iclamp.dur = 50 # ms

In [ ]:
neuron.h.run()

plot_tv(time, voltage)

## 6. Adding a leak conductance

We'll now start adding transmembrane currents to the model as described in the **Hodgkin-Huxley model**.
To enable the equations for this model, we insert the 'hh' mechanism in the soma:

In [ ]:
soma.insert('hh');

Now NEURON will integrate the following equation to calculate the membrane voltage over time:

$$C_{m} dV/dt = -(I_{Na} + I_{K} + I_{leak}) + I_{ext} = -(g_{Na}(V)*(V - E_{Na}) + g_{K}(V)*(V - E_{K}) + g_{leak}*(V - E_{leak})) + I_{ext}$$

We have a sodium and potassium current with voltage dependent conductance, and a leakage current with a voltage independent conductance.

Initially we'll set the Na and K conductance to zero:

In [ ]:
soma.gkbar_hh = 0.0
soma.gnabar_hh = 0.0

The value for the leak conductance is set using the 'gl_hh' attribute ('hh' refers to the name of the mechanism, 'gl' to the name of the parameter inside the 'hh' description). 

In [ ]:
soma.gl_hh = 5e-4 # Leak conductance, S/cm^2

The 'el_hh' attribute sets the reversal potential. We call this the 'reversal' potential because the value corresponds to the membrane voltage at which the leak current changes polarity. It's the potential towards which this type of the channel 'pushes' the membrane voltage. Let's set the value equal to the resting membrane potential of the cell, so that we have a force that tries to maintain -65 mV.

In [ ]:
el = soma.el_hh = -65 # Reversal potential leak current, mV
print "Reveral of leak current: %f mV" % el

In [ ]:
soma.gkbar_hh = 0.0
soma.gnabar_hh = 0.0

neuron.h.tstop = 100

neuron.h.run()

plot_tv(time, voltage)

## 7. Adding active ion channels

Now we are adding the active voltage dependent ion channels of the Hodgkin-Huxley mechanism.

$$C_{m} \frac{dV}{dt} = -(g_{Na}(V)(V - E_{Na}) + g_{K}(V)(V - E_{K}) + g_{leak}(V - E_{leak})) + I_{ext} = -({\overline{g}}_{Na}m^{3}h(V - E_{Na}) + {\overline{g}}_{K}n^{4}(V - E_{K}) + g_{leak}(V - E_{leak})) + I_{ext}$$

The conductance $g_{Na}$ and $g_{K}$ are decomposed in $\overline{g}_{Na} m^3h$ and $\overline{g}_{K} n^4$. The $\overline{g}_{Na}$ and $\overline{g}_{K}$ are parameters of the model that are expressed in siemens/cm^2. They basically represent the density of the ion channels on the membrane. Let's set these to a value different than 0.
When we run the model, we will see that the model now generates action potentials.

In [ ]:
soma.gkbar_hh = 0.01 # in S/cm^2
soma.gnabar_hh = 0.1 # in S/cm^2

neuron.h.run()

plot_tv(time, voltage)